In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn import metrics as metrics
from sklearn.model_selection import train_test_split, ParameterGrid


from sklearn.dummy import DummyClassifier

# Import dataframes

In [2]:
df_train = pd.read_csv('data/train.csv', header=0) # -> treino
df_test = pd.read_csv('data/test.csv', header=0) # -> teste

# Feature engineering and data preparation
    - Not considering NaN still

In [3]:
# Define the WIP DataFrame for backup
df_full = pd.concat([df_train.drop(columns=['Transported']), df_test]).reset_index(drop=True)


wip_df = df_full.copy()

In [4]:
display(wip_df.head())
display(wip_df.info())
wip_df.describe()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12660 non-null  object 
 3   Cabin         12671 non-null  object 
 4   Destination   12696 non-null  object 
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  object 
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Name          12676 non-null  object 
dtypes: float64(6), object(7)
memory usage: 1.3+ MB


None

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,12700.000000,12707.000000,12681.000000,12664.000000,12686.000000,12702.000000
mean,28.771969,222.897852,451.961675,174.906033,308.476904,306.789482
std,14.387261,647.596664,1584.370747,590.558690,1130.279641,1180.097223
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,49.000000,77.000000,29.000000,57.000000,42.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


- Working with PassengerId
    - Separating Passenger groups within passengers

In [5]:
def get_passenger_group(passengerid: str) -> str:
    if re.search('_', passengerid):
        return passengerid[0:4]
    
    else:
        return 'check'
    
def set_passenger_groupsize(passengergroup: int, grouped_df: pd.DataFrame) -> str:
    if grouped_df['PassengerId'][passengergroup] == 1:
        return 'Alone'
    
    elif grouped_df['PassengerId'][passengergroup] == 2:
        return 'Pair'
    
    elif grouped_df['PassengerId'][passengergroup] >= 3:
        return 'Family'

In [6]:
wip_df['PassengerGroup'] = wip_df.apply(lambda x: get_passenger_group(x['PassengerId']), axis=1)
# wip_df.loc[wip_df['PassengerGroup'] == 'check'] <- no check found

In [7]:
grouped_df = wip_df[['PassengerGroup','PassengerId']].groupby('PassengerGroup').count()
wip_df['Riding'] = wip_df.apply(lambda x: set_passenger_groupsize(x['PassengerGroup'], grouped_df), axis=1)

In [8]:
wip_df.sample(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,PassengerGroup,Riding
8876,0374_01,Earth,False,F/86/P,TRAPPIST-1e,18.0,False,0.0,251.0,NaN,0.0,553.0,Harrie Sterreray,0374,Pair
7247,7748_02,Europa,False,A/94/S,TRAPPIST-1e,39.0,True,0.0,395.0,0.0,1012.0,1080.0,Almino Obsentork,7748,Pair
10394,3646_01,Europa,False,C/131/S,TRAPPIST-1e,16.0,False,7.0,1879.0,0.0,28.0,2186.0,Alcorix Drelcate,3646,Pair
7548,8068_01,Earth,False,F/1545/S,TRAPPIST-1e,20.0,False,0.0,52.0,0.0,110.0,614.0,Eduard Mcfarmerson,8068,Alone
10850,4683_01,Mars,True,F/953/P,TRAPPIST-1e,38.0,False,0.0,0.0,0.0,0.0,0.0,Honjay Sacke,4683,Pair
9927,2614_01,Mars,False,F/503/S,TRAPPIST-1e,25.0,False,938.0,0.0,206.0,0.0,289.0,Cruses Boure,2614,Alone
6871,7271_01,Earth,False,G/1171/P,55 Cancri e,21.0,False,61.0,201.0,222.0,161.0,60.0,NaN,7271,Alone
329,0370_01,Earth,False,F/83/P,TRAPPIST-1e,30.0,False,0.0,95.0,0.0,441.0,1023.0,Tine Coffmaney,0370,Alone
2020,2158_01,Earth,False,F/423/S,TRAPPIST-1e,21.0,False,0.0,820.0,0.0,45.0,0.0,Joyn Moraleney,2158,Alone
8356,8940_01,Earth,False,E/587/S,TRAPPIST-1e,24.0,False,771.0,0.0,22.0,0.0,0.0,NaN,8940,Alone


- Working with RoomService, FoodCourt, ShoppingMall, Spa and VRDeck bills
    - Resolving the problem with NaNs
    - Filling all other NaN with 0

In [9]:
columns = [
    'RoomService',
    'FoodCourt',
    'ShoppingMall',
    'Spa',
    'VRDeck'
]

wip_df['TotalBill'] = wip_df[columns].sum(axis=1)
wip_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,PassengerGroup,Riding,TotalBill
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0001,Alone,0.0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,0002,Alone,736.0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0003,Pair,10383.0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0003,Pair,5176.0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,0004,Alone,1091.0


In [10]:
wip_df[columns] = wip_df[columns].fillna(0)

- Working with Destination Objects
    - TRAPPIST-1e
    - 55 Cancri e
    - PSO J318.5-22

In [11]:
wip_df['Destination'] = wip_df['Destination'].fillna('X')
wip_df['Destination'] = pd.Categorical(wip_df['Destination'])

In [12]:
wip_df['Destination'].value_counts(dropna=False)

TRAPPIST-1e      8871
55 Cancri e      2641
PSO J318.5-22    1184
X                 274
Name: Destination, dtype: int64

- Working with Cabins
    - Splitting into Deck/Side

In [13]:
wip_df['Cabin'] = wip_df['Cabin'].fillna('X/0/X')
wip_df[['Cabin_Deck','Cabin_Number','Cabin_Side']] = wip_df['Cabin'].str.split('/', expand=True)

In [14]:
wip_df['Cabin_Deck'] = pd.Categorical(wip_df['Cabin_Deck'])
wip_df['Cabin_Side'] = pd.Categorical(wip_df['Cabin_Side'])

- Working with CryoSleep
    - Transforming column to bool ignoring NaN

In [15]:
wip_df['CryoSleep'] = wip_df['CryoSleep'].astype(bool)

- Working with VIP
    - Transforming column to bool ignoring NaN

In [16]:
wip_df['VIP'] = wip_df['VIP'].astype(bool)

- Working with HomePlanet
    - Europa, Mars, Earth and Empty HomePlanets

In [17]:
wip_df['HomePlanet'] = wip_df['HomePlanet'].fillna('X')
wip_df['HomePlanet'] = pd.Categorical(wip_df['HomePlanet'])

- Working with Age
    - Filling NaN with median

In [18]:
wip_df['Age'] = wip_df.Age.fillna(wip_df.Age.median())

In [19]:
wip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   PassengerId     12970 non-null  object  
 1   HomePlanet      12970 non-null  category
 2   CryoSleep       12970 non-null  bool    
 3   Cabin           12970 non-null  object  
 4   Destination     12970 non-null  category
 5   Age             12970 non-null  float64 
 6   VIP             12970 non-null  bool    
 7   RoomService     12970 non-null  float64 
 8   FoodCourt       12970 non-null  float64 
 9   ShoppingMall    12970 non-null  float64 
 10  Spa             12970 non-null  float64 
 11  VRDeck          12970 non-null  float64 
 12  Name            12676 non-null  object  
 13  PassengerGroup  12970 non-null  object  
 14  Riding          12970 non-null  object  
 15  TotalBill       12970 non-null  float64 
 16  Cabin_Deck      12970 non-null  category
 17  Cabin_Number

- Getting the dummies from cols

In [20]:
dummy_cols = ['HomePlanet','Destination','Riding','Cabin_Deck','Cabin_Side']

dummies = pd.get_dummies(wip_df[['PassengerId','HomePlanet','Destination','Riding','Cabin_Deck','Cabin_Side']], dummy_na=True, columns=dummy_cols)

In [21]:
# Adding dummies and saving final DataFrame
df_full = wip_df.drop(columns=
    dummy_cols + 
    [
        'Cabin',
        'Name',
        'PassengerGroup',
        'Cabin_Number'
    ]
)

In [22]:
df_full = df_full.merge(dummies, on='PassengerId', how='inner')

# Model selection

In [23]:
# Returning the preprocessed DataFrame to its original values
df_train = df_train[['PassengerId','Transported']].merge(df_full, on='PassengerId', how='inner').copy()
df_test = df_test[['PassengerId']].merge(df_full, on='PassengerId', how='inner').copy()

df_train = df_train.drop(columns='PassengerId')
df_test = df_test.drop(columns='PassengerId')

In [24]:
x = df_train.drop(columns='Transported')
y = df_train['Transported']

In [25]:
# Testing if dataset is valid to training
dm = DummyClassifier(random_state=28)
dm.fit(x, y)

dm_predictions = dm.predict(x)
metrics.accuracy_score(y, dm_predictions)

0.5036236051995858

- Train, Test, Split

In [26]:
train_x, val_x, train_y, val_y = train_test_split(x, y, train_size = 0.6, random_state=28)

- Checking Accuracy from different models

In [44]:
models = [
    LinearSVC(random_state=28),
    KNeighborsClassifier(),
    GaussianNB(),
    RandomForestClassifier(random_state=28),
    AdaBoostClassifier(random_state=28),
    SVC(random_state=28)
]

for model in models:
    model.fit(train_x, train_y)

    model_preds = model.predict(val_x)
    print(f'Model: {model}, ACC: {metrics.accuracy_score(val_y, model_preds)}')

c:\Users\covin\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model: LinearSVC(random_state=28), ACC: 0.7380678550891316
Model: KNeighborsClassifier(), ACC: 0.7708453133985049
Model: GaussianNB(), ACC: 0.7366302472685451
Model: RandomForestClassifier(random_state=28), ACC: 0.7880966072455434
Model: AdaBoostClassifier(random_state=28), ACC: 0.7826336975273146
Model: SVC(random_state=28), ACC: 0.7886716503737781


In [43]:
np.linspace(1,2,11)

array([1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ])